<a href="https://colab.research.google.com/github/Matrixmax/Dive_into_DeepLearning/blob/main/0x05_%E8%87%AA%E5%8A%A8%E6%B1%82%E5%AF%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 一个简单的例子

In [1]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算 y 关于 x 的梯度之前，我们需要一个地方来存储梯度。 重要的是，我们不会在每次对一个参数求导时都分配新的内存。因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。注意，标量函数关于向量 x 的梯度是向量，并且与 x 具有相同的形状。

In [2]:
x.requires_grad_(True) # 等价于 `x = torch.arange(4.0, requires_grad=True)`
x.grad # 默认值是None

In [4]:
y = 2*torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

函数  y=2x⊤x  关于 x  的梯度应为 4x 。让我们快速验证我们想要的梯度是否正确计算。

In [6]:
x.grad == 4*x

tensor([True, True, True, True])

现在让我们计算 x 的另一个函数。

In [10]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 2. 非标量变量的反向传播

当 y 不是标量时，向量y关于向量x的导数的最自然解释是一个矩阵。对于高阶和高维的 y 和 x，求导的结果可以是一个高阶张量。

In [12]:
# 对非标量调用`backward`需要传入一个`gradient`参数，该参数指定微分函数关于`self`的梯度。
# 在我们的例子中，我们只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x*x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## 3. 分离计算
有时，我们希望将某些计算移动到记录的计算图之外。 例如，假设y是作为x的函数计算的，而z则是作为y和x的函数计算的。 现在，想象一下，我们想计算 z 关于 x 的梯度，但由于某种原因，我们希望将 y 视为一个常数，并且只考虑到 x 在y被计算后发挥的作用。

在这里，我们可以分离 y 来返回一个新变量 u，该变量与 y 具有相同的值，但丢弃计算图中如何计算 y 的任何信息。换句话说，梯度不会向后流经 u 到 x。因此，下面的反向传播函数计算 z = u * x 关于 x 的偏导数，同时将 u 作为常数处理，而不是z = x * x * x关于 x 的偏导数。

In [13]:
x.grad.zero_()
y = x*x
u = y.detach()
z = u*x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

由于记录了 y 的计算结果，我们可以随后在 y 上调用反向传播，得到 y = x * x 关于的x的导数，这里是 2 * x。

In [14]:
x.grad.zero_()
y.sum().backward()
x.grad ==2*x

tensor([True, True, True, True])

## 4. python控制流的梯度计算
使用自动求导的一个好处是，即使构建函数的计算图需要通过 Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。在下面的代码中，while 循环的迭代次数和 if 语句的结果都取决于输入 a 的值。

In [17]:
def f(a):
    b = a*2
    while b.norm() < 1000:
        b = b*2
    if b.sum()>0:
        c = b
    else:
        c = 100*b
    return c

In [18]:
a = torch.randn(size=(),requires_grad=True)
d = f(a)
d.backward()

In [19]:
a.grad == d/a

tensor(True)